In [ ]:
# https://www.baozimh.com/

In [13]:
import requests
import re
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin
from PIL import Image
import time
from io import BytesIO

    
def getComicMaxNum(url="https://www.baozimh.com/comic/diyuchuangzaozhe-nailuoyounggi"):
    # 網頁的URL
#     url = 'https://www.baozimh.com/comic/diyuchuangzaozhe-nailuoyounggi'

    # 使用requests庫發送GET請求並獲取網頁內容
    response = requests.get(url, timeout=30)

    # 確保成功獲取網頁內容
    if response.status_code == 200:
        # 使用lxml解析器解析HTML內容
        soup = BeautifulSoup(response.text, 'lxml')

        # 找到title標籤並印出其內容
        title = soup.title.string
        title = title[1:-7]
        print("漫畫標題：", title)
        if os.path.exists("example"):
            pass
        else:
            os.mkdir("example")
        # 包含話數的HTML片段
        html = response.text
        # 使用正則表達式提取href中的數字
        chapter_numbers = re.findall(r'chapter_slot=(\d+)', html)

        # 將提取的數字轉換為整數
        chapter_numbers = [int(num) for num in chapter_numbers]

        # 找出最大的數字
        max_chapter_number = max(chapter_numbers)

        print("所有話數中的最大數字是:", max_chapter_number)
        return title,max_chapter_number

def DownloadImages(comic_name,url,chapter_num):
    os.mkdir(f"./example/Chapter")
    # 目標網址
    url = f"https://www.dociy.net/comic/chapter/{comic_name}/0_{chapter_num}.html"  # 請替換為你的目標網址

    # 發送GET請求獲取頁面內容
    response = requests.get(url)

    # 使用Beautiful Soup解析HTML頁面
    soup = BeautifulSoup(response.text, "html.parser")

    # 找到第一個jpg圖片
    jpg_image = soup.find("img", src=lambda x: x and x.endswith(".jpg"))

    if jpg_image:
        img_url = jpg_image["src"]
        print("第一個JPG圖片的來源網址:", img_url)
        value = img_url.split("/")
        random_str = value[-2][-4:]

    for img_num in range(1,101):
        try:
            # 要下載的圖片網址
            image_url = f"https://s1.baozimh.com/scomic/{comic_name}/0/{chapter_num}-{random_str}/{img_num}.jpg"
            print(image_url)
#             time.sleep(1)
            # 發送GET請求獲取圖片內容
            response = requests.get(image_url)

            # 確保響應狀態碼是200（成功）
            if response.status_code == 200:
                # 將圖片內容轉換為BytesIO對象
                image_data = BytesIO(response.content)

                # 使用Pillow打開圖片
                image = Image.open(image_data)

                # 保存圖片到本地
                image.save(f"./example/Chapter/{img_num}.jpg")
#                 image.save(f"./example/Chapter_{chapter_num}/{img_num}.jpg")
                print(f"圖片下載成功並保存為./{title}/{chapter_num}/{img_num}.jpg")
#             else:
#                 print("無法下載圖片。響應狀態碼:", response.status_code)
        except Exception as e:
#             print("發生錯誤:", str(e))
            pass

    # 舊的資料夾名稱
    old_folder_name = "example/Chapter"

    # 新的資料夾名稱
    new_folder_name = f"example/Chapter_{chapter_num}"

    try:
        # 使用os.rename函數來修改資料夾名稱
        os.rename(old_folder_name, new_folder_name)
        print(f"成功將資料夾名稱從 {old_folder_name} 修改為 {new_folder_name}")
    except Exception as e:
        print(f"修改資料夾名稱時出現錯誤: {str(e)}")

    mergeImages(chapter_num)
        
def mergeImages(c_num):
    if os.path.exists(f"./example/MergeImage"):
        pass
    else:
        os.mkdir(f"./example/MergeImage")
    # 資料夾路徑，包含要合併的圖片
    folder_path = f"./example/Chapter_{c_num}"
    if os.path.exists(f"./example/MergeImage/merged_image Chapter {c_num}.png"):
        pass
    else:
        # 獲取資料夾中的所有圖片檔案列表
        image_files = [f for f in os.listdir(folder_path) if f.endswith(".jpg")]

        # 按照檔名順序由小到大排序
        image_files.sort(key=lambda x: int(x.split('.')[0]))

        # 創建一個空白圖片，高度為所有圖片高度的總和，寬度為第一張圖片的寬度
        first_image = Image.open(os.path.join(folder_path, image_files[0]))
        result_image = Image.new("RGB", (first_image.width, sum(Image.open(os.path.join(folder_path, img)).height for img in image_files)))

        # 將每張圖片逐一粘貼到結果圖片上
        y_offset = 0
        for img_file in image_files:
            img = Image.open(os.path.join(folder_path, img_file))
            result_image.paste(img, (0, y_offset))
            y_offset += img.height

        # 儲存合併後的圖片
        result_image.save(f"./example/MergeImage/merged_image Chapter {c_num}.png")


        print(f"圖片合併完成並已儲存為 merged_image {c_num}.png")

if __name__ == "__main__":
    print("請輸入包子漫畫網址\n例如:https://www.baozimh.com/comic/diyuchuangzaozhe-nailuoyounggi\n")
    url = input()
    # url="https://www.baozimh.com/comic/diyuchuangzaozhe-nailuoyounggi"
    a=url.split("/")
    comic_name = a[-1]
    title,max_num = getComicMaxNum(url)
#     title,max_num = getComicMaxNum(url = "https://www.baozimh.com/comic/diyuchuangzaozhe-nailuoyounggi")
    


    for c_num in range(max_num+1):
        DownloadImages(comic_name,url,c_num)
    # 舊的資料夾名稱
    old_folder_name = "example"

    # 新的資料夾名稱
    new_folder_name = title

    try:
        # 使用os.rename函數來修改資料夾名稱
        os.rename(old_folder_name, new_folder_name)
        print(f"成功將資料夾名稱從 {old_folder_name} 修改為 {new_folder_name}")
    except Exception as e:
        print(f"修改資料夾名稱時出現錯誤: {str(e)}")

請輸入包子漫畫網址
例如:https://www.baozimh.com/comic/diyuchuangzaozhe-nailuoyounggi

https://www.baozimh.com/comic/diyuchuangzaozhe-nailuoyounggi
漫畫標題： 地獄創造者
所有話數中的最大數字是: 53
第一個JPG圖片的來源網址: https://s1.baozicdn.com/scomic/diyuchuangzaozhe-nailuoyounggi/0/0-vkeu/1.jpg
https://s1.baozimh.com/scomic/diyuchuangzaozhe-nailuoyounggi/0/0-vkeu/1.jpg
圖片下載成功並保存為./地獄創造者/0/1.jpg
https://s1.baozimh.com/scomic/diyuchuangzaozhe-nailuoyounggi/0/0-vkeu/2.jpg


KeyboardInterrupt: 